In [ ]:
from __future__ import annotations
import os, sys
import pandas as pd
from pathlib import Path
import numpy as np
from Bio import SeqIO
from Bio.Seq import Seq
from local.constants import WORKSPACE_ROOT as HERE

os.makedirs("./cache", exist_ok=True)

class RawSeq:
    def __init__(self, seq: Seq|str) -> None:
        self._seq = Seq(seq) if isinstance(seq, str) else seq

    def __len__(self) -> int:
        return len(self._seq)
    
    def __str__(self) -> str:
        return str(self._seq)
    
    def __repr__(self) -> str:
        return self.__str__()
    
    def ReverseCompliment(self) -> str:
        x = RawSeq(str(self._seq.reverse_complement()))
        self._seq.reverse_complement()
        return str(x)

In [2]:
np.random.seed(12345)
ALPHABET = "ACGT"
INSERT_LEN = 40_000

END_LEN = 600
CUTT_END = 1000
SHORT_LEN = 30

class TestCases:
    @classmethod
    def generaete_insert(cls):
        return RawSeq("".join(ALPHABET[i] for i in np.random.choice(len(ALPHABET), INSERT_LEN, replace=True)))

    def case_perfect(self, s: RawSeq):
        fwd = str(s)[:END_LEN]
        rev = s.ReverseCompliment()[:END_LEN]
        return fwd, rev, [s], s

    def case_perfect_rc(self, s: RawSeq):
        fwd = str(s)[:END_LEN]
        rev = s.ReverseCompliment()[:END_LEN]
        return fwd, rev, [s.ReverseCompliment()], s

    def case_assembly_slightly_short(self, s: RawSeq):
        fwd = str(s)[:END_LEN]
        rev = s.ReverseCompliment()[:END_LEN]
        return fwd, rev, [str(s)[SHORT_LEN:len(s)-SHORT_LEN]], s

    def case_assembly_slightly_short_rc(self, s: RawSeq):
        fwd = str(s)[:END_LEN]
        rev = s.ReverseCompliment()[:END_LEN]
        return fwd, rev, [s.ReverseCompliment()[SHORT_LEN:len(s)-SHORT_LEN]], s

    def case_facing_but_past(self, s: RawSeq):
        fwd = str(s)[:END_LEN]
        rev = s.ReverseCompliment()[:END_LEN]
        fwd = str(RawSeq(fwd).ReverseCompliment())
        rev = str(RawSeq(str(rev)).ReverseCompliment())
        return fwd, rev, [s], fwd # essentially no match

    def case_needs_scaffold(self, s: RawSeq):
        fwd = str(s)[:END_LEN]
        rev = s.ReverseCompliment()[:END_LEN]
        mid = len(s)//2
        SCAFFOLD_OVERLAP = 1000
        return fwd, rev, [str(s)[:mid+(SCAFFOLD_OVERLAP//2)], str(s)[(mid-SCAFFOLD_OVERLAP//2):]], s

    def case_needs_scaffold_off_center(self, s: RawSeq):
        fwd = str(s)[:END_LEN]
        rev = s.ReverseCompliment()[:END_LEN]
        SCAFFOLD_OVERLAP = 2000
        mid = (len(s)//2)+5_000
        return fwd, rev, [str(s)[:mid+SCAFFOLD_OVERLAP//2], str(s)[mid-SCAFFOLD_OVERLAP//2:]], s
    
    def case_needs_scaffold_rc(self, s: RawSeq):
        fwd = str(s)[:END_LEN]
        rev = s.ReverseCompliment()[:END_LEN]
        mid = len(s)//2
        SCAFFOLD_OVERLAP = 1000
        return fwd, rev, [str(s)[:mid+SCAFFOLD_OVERLAP//2], s.ReverseCompliment()[:mid+SCAFFOLD_OVERLAP//2]], s

    def case_needs_scaffold_slightly_short_rc(self, s: RawSeq):
        fwd = str(s)[:END_LEN]
        rev = s.ReverseCompliment()[:END_LEN]
        mid = len(s)//2
        SCAFFOLD_OVERLAP = 1000
        return fwd, rev, [str(s)[SHORT_LEN:mid+SCAFFOLD_OVERLAP//2], s.ReverseCompliment()[SHORT_LEN:mid+SCAFFOLD_OVERLAP//2]], s
    
    def case_fwd_only(self, s: RawSeq):
        fwd = str(s)[:END_LEN]
        rev = s.ReverseCompliment()[:END_LEN]
        ss = str(s)[:len(s)-CUTT_END]
        return fwd, rev, [ss], ss
    
    def case_rev_only(self, s: RawSeq):
        fwd = str(s)[:END_LEN]
        rev = s.ReverseCompliment()[:END_LEN]
        ss = str(s)[CUTT_END:]
        return fwd, rev, [ss], ss
    
    def case_fwd_missing(self, s: RawSeq):
        rev = s.ReverseCompliment()[:END_LEN]
        return None, rev, [s], s
    
    def case_rev_missing(self, s: RawSeq):
        fwd = str(s)[:END_LEN]
        return fwd, None, [s], s
    
    def case_fwd_only_rc(self, s: RawSeq):
        fwd = str(s)[:END_LEN]
        rev = s.ReverseCompliment()[:END_LEN]
        ss = str(s)[:len(s)-CUTT_END]
        return fwd, rev, [RawSeq(ss).ReverseCompliment()], ss
    
    def case_rev_only_rc(self, s: RawSeq):
        fwd = str(s)[:END_LEN]
        rev = s.ReverseCompliment()[:END_LEN]
        ss = str(s)[CUTT_END:]
        return fwd, rev, [RawSeq(ss).ReverseCompliment()], ss
    
    def case_fwd_only_slightly_short(self, s: RawSeq):
        fwd = str(s)[:END_LEN]
        rev = s.ReverseCompliment()[:END_LEN]
        ss = str(s)[:len(s)-CUTT_END]
        return fwd, rev, [ss[SHORT_LEN:]], ss
    
    def case_rev_only_slightly_short(self, s: RawSeq):
        fwd = str(s)[:END_LEN]
        rev = s.ReverseCompliment()[:END_LEN]
        ss = str(s)[CUTT_END:]
        return fwd, rev, [ss[:len(s)-SHORT_LEN]], ss
    
    def case_fwd_only_slightly_short_rc(self, s: RawSeq):
        fwd = str(s)[:END_LEN]
        rev = s.ReverseCompliment()[:END_LEN]
        ss = str(s)[:len(s)-CUTT_END]
        return fwd, rev, [RawSeq(ss).ReverseCompliment()[:-SHORT_LEN]], ss
    
    def case_rev_only_slightly_short_rc(self, s: RawSeq):
        fwd = str(s)[:END_LEN]
        rev = s.ReverseCompliment()[:END_LEN]
        ss = str(s)[CUTT_END:]
        return fwd, rev, [RawSeq(ss).ReverseCompliment()[SHORT_LEN:]], ss

expected = {}
cases = []
fwd_ends, rev_ends, contigs = "./cache/fwds.fa", "./cache/revs.fa", "./cache/contigs.fa"
with open(fwd_ends, "w") as fw, open(rev_ends, "w") as rv, open(contigs, "w") as con:
    for k, v in TestCases.__dict__.items():
        if not k.startswith("case"): continue
        cases.append(k)
        test_seq = TestCases.generaete_insert()
        _fwd, _rev, _assemblies, _expected = v(TestCases, test_seq)
        if _fwd:
            fw.write(f">{k}\n{_fwd}\n")
        if _rev:
            rv.write(f">{k}\n{_rev}\n")
        for i, s in enumerate(_assemblies):
            con.write(f">{k}_{i}\n{s}\n")
        expected[k] = RawSeq(str(_expected))

In [38]:
ENV = Path(os.environ["CONDA_PREFIX"])/"../fabfos"
ENV = ENV.resolve().absolute()

test_results_dir = "./cache/test_scaffold"
os.system(f"""\
    export PYTHONPATH={HERE}/src:$PYTHONPATH
    export PATH={ENV}/bin:$PATH
    {ENV}/bin/python -m fabfos run \
        --overwrite \
        --exp_length 40000 \
        --exp_length_range 2000 \
        --gap_str N \
        -t 14 \
        -a {contigs} \
        --endf {fwd_ends} \
        --endr {rev_ends} \
        --ends_facing \
        --end_regex "\\w+" \
        -o {test_results_dir} \
""")

Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 14
Rules claiming more threads will be scaled down.
Job stats:
job                count
---------------  -------
acquire_contigs        1
scaffold               1
target                 1
total                  3

Select jobs to execute...

[Wed Mar 13 16:51:04 2024]
rule acquire_contigs:
    input: internals/temp_reads/original_reads.json, internals/temp_assembly/assemblies.json
    output: internals/temp_assembly/contigs.json
    jobid: 2
    reason: Forced execution
    threads: 14
    resources: tmpdir=/tmp

Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 14
Rules claiming more threads will be scaled down.
Job stats:
job                count
---------------  -------
acquire_contigs        1
scaffold               1
target                 1
total                  3

Select jobs to execute...

[Wed Mar 13 16:51:04 2024]
rule acquire_contigs:
    input: internals/temp_reads/original_reads.json, inter

0

In [29]:
actual = {s.id:s.seq for s in SeqIO.parse(f"{test_results_dir}/scaffolds.fna", "fasta")}
len(actual), len(expected)

(19, 19)

In [39]:
dfm = pd.read_csv(f"{test_results_dir}/mapping_report.csv")
dfm

,id,mapping_quality,paired,resolved_length,assemblers,contig_id,original_id
0,case_assembly_slightly_short,full_match,True,40000,given_contigs,C03,case_assembly_slightly_short_0
1,case_assembly_slightly_short_rc,full_match,True,40000,given_contigs,C04,case_assembly_slightly_short_rc_0
2,case_facing_but_past,forward_end_only,False,600,given_contigs,C05,case_facing_but_past_0
3,case_fwd_missing,reverse_end_only,False,40000,given_contigs,C16,case_fwd_missing_0
4,case_fwd_only,forward_end_only,False,39000,given_contigs,C14,case_fwd_only_0
5,case_fwd_only_rc,forward_end_only,False,39000,given_contigs,C18,case_fwd_only_rc_0
6,case_fwd_only_slightly_short,forward_end_only,False,39000,given_contigs,C20,case_fwd_only_slightly_short_0
7,case_fwd_only_slightly_short_rc,forward_end_only,False,39000,given_contigs,C22,case_fwd_only_slightly_short_rc_0
8,case_needs_scaffold,full_scaffold,True,40000,given_contigs;given_contigs,C06;C07,case_needs_scaffold_0;case_needs_scaffold_1
9,case_needs_scaffold_off_center,full_scaffold,True,40000,given_contigs;given_contigs,C08;C09,case_needs_scaffold_off_center_0;case_needs_sc...


In [30]:
_err = False
for k in cases:
    if k not in actual:
        print("missing", k)
        _err = True

    e, a = [str(s) for s in [expected[k], actual[k]]]
    if e != a:
        print("unequal", k)
        print(len(e)-len(a))
        _err = True
if not _err:
    print("all tests passed")

all tests passed


,id,mapping_quality,paired,resolved_length,assemblers,contig_id,original_id
0,case_assembly_slightly_short,full_match,True,40000,contigs,C03,case_assembly_slightly_short_0
1,case_assembly_slightly_short_rc,full_match,True,40000,contigs,C04,case_assembly_slightly_short_rc_0
2,case_facing_but_past,forward_end_only,False,600,contigs,C05,case_facing_but_past_0
3,case_fwd_missing,reverse_end_only,False,40000,contigs,C16,case_fwd_missing_0
4,case_fwd_only,forward_end_only,False,39000,contigs,C14,case_fwd_only_0
5,case_fwd_only_rc,forward_end_only,False,39000,contigs,C18,case_fwd_only_rc_0
6,case_fwd_only_slightly_short,forward_end_only,False,39000,contigs,C20,case_fwd_only_slightly_short_0
7,case_fwd_only_slightly_short_rc,forward_end_only,False,39000,contigs,C22,case_fwd_only_slightly_short_rc_0
8,case_needs_scaffold,full_scaffold,True,40000,contigs;contigs,C06;C07,case_needs_scaffold_0;case_needs_scaffold_1
9,case_needs_scaffold_off_center,full_scaffold,True,40000,contigs;contigs,C08;C09,case_needs_scaffold_off_center_0;case_needs_sc...


In [31]:
k = "case_fwd_only"
print(k)
print(len(expected[k]), len(actual[k]), len(expected[k])-len(actual[k]))
print(expected[k])
print(actual[k])
print(type(actual[k]), type(expected[k]))

case_fwd_only
39000 39000 0
...
<class 'Bio.Seq.Seq'> <class '__main__.RawSeq'>


In [23]:
RawSeq("CAACTTCTCAAATGGACCCGGTCTTTACT").ReverseCompliment()

'AGTAAAGACCGGGTCCATTTGAGAAGTTG'

In [24]:
x = "0123456789"
s, e = 5, 1
x[e:s], x[::-1][len(x)-s:len(x)-e]

('1234', '4321')

In [25]:
# different results from spades 3.15.5 vs 3.15.4 (Joe)
# do ablation plot of different assemblers